# Assistente Avaliador de Notícias

## Descrição do Projeto
Este agente de IA é capaz de:
1. **Buscar notícias** na web usando DuckDuckGo
2. **Avaliar o sentimento** das notícias (positivo ou negativo)
3. **Monitorar tokens** utilizados em cada operação

## Dependências
As dependências necessárias podem ser instaladas pelo `requirements.txt` ou pelo bloco abaixo

In [103]:
%pip install --upgrade \
    python-dotenv \
    langchain \
    langchain-core \
    langchain-mistralai \
    langchain-community \
    duckduckgo-search \
    langgraph \
    transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


##### Imports

In [104]:
import os
from typing import Dict, Any
from langchain_mistralai import ChatMistralAI
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import PromptTemplate
from langchain.tools import tool

load_dotenv()

True

##### API_KEY
Adquira a chave no site https://admin.mistral.ai/organization/api-keys se necessário

In [105]:
api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("MISTRAL_API_KEY não encontrada. Configure no arquivo .env")

Definindo modelo LLM

In [106]:
llm = ChatMistralAI(
    model="open-mistral-7b",
    temperature=0,
    max_retries=1,
    verbose=True # Ajuda a ver tokens e modelo
)

## 1. Tool de Busca de Notícias

Implementação da tool que busca notícias usando DuckDuckGo

In [107]:
@tool
def buscar_noticias(query: str) -> str:
    """Busca notícias na web sobre um tópico específico usando DuckDuckGo."""
    try:
        search = DuckDuckGoSearchRun()
        resultados = search.run(query)
        return resultados
    except Exception as e:
        return f"Erro ao buscar notícias: {str(e)}"

print(f"Tool '{buscar_noticias.name}' criada com sucesso!")

Tool 'buscar_noticias' criada com sucesso!


## 2. Tool de Análise de Sentimento

Implementação da tool que avalia se o conteúdo é positivo ou negativo

In [ ]:
@tool
def analisar_sentimento(texto: str) -> str:
    """Analisa o sentimento de um texto e determina se é positivo ou negativo. Use após obter o conteúdo de notícias."""
    try:
        prompt = PromptTemplate(
            input_variables=["text"],
            template="""Analise o sentimento do seguinte texto de uma notícia e classifique como POSITIVO ou NEGATIVO. Use Português Brasileiro e de prefêrencia conteúdo produzido no Brasil

Texto: {text}

Responda no formato:
NOTICIAS: [Coloque o titulo das noticias e nome do jornal que as publicou]
SENTIMENTO: [POSITIVO/NEGATIVO]
JUSTIFICATIVA: [explicação breve do porquê]
"""
        )
        
        chain = prompt | llm
        resultado = chain.invoke({"text": texto})
        
        # Extrair o conteúdo da resposta
        if hasattr(resultado, 'content'):
            return resultado.content
        return str(resultado)
    except Exception as e:
        return f"Erro ao analisar sentimento: {str(e)}"

print(f"Tool '{analisar_sentimento.name}' criada com sucesso!")

Tool 'analisar_sentimento' criada com sucesso!


## 3. Configuração do Agente com Monitoramento de Tokens

Criação do agente ReAct com as tools e sistema de contagem de tokens

In [109]:
# Classe para monitorar tokens
class TokenMonitor:
    def __init__(self):
        self.total_tokens = 0
        self.operacoes = []
    
    def adicionar_operacao(self, operacao: str, tokens: int):
        self.total_tokens += tokens
        self.operacoes.append({"operacao": operacao, "tokens": tokens})
    
    def obter_relatorio(self):
        relatorio = f"\n{'='*50}\n"
        relatorio += "RELATÓRIO DE USO DE TOKENS\n"
        relatorio += f"{'='*50}\n"
        for op in self.operacoes:
            relatorio += f"{op['operacao']}: {op['tokens']} tokens\n"
        relatorio += f"{'='*50}\n"
        relatorio += f"TOTAL: {self.total_tokens} tokens\n"
        relatorio += f"{'='*50}\n"
        return relatorio

# Inicializar monitor
token_monitor = TokenMonitor()

# Criar lista de tools
tools = [buscar_noticias, analisar_sentimento]

# Criar um agente simples que chama as tools conforme necessário
class SimpleNewsAgent:
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
    
    def run(self, query: str) -> str:
        """Executa o agente para processar a query"""
        # Primeiro, busca notícias
        print("\nBuscando notícias...")
        news_result = self.tools['buscar_noticias'].invoke(query)
        
        # Depois, analisa o sentimento
        print("\nAnalisando sentimento...")
        sentiment_result = self.tools['analisar_sentimento'].invoke(news_result[:2000])  # Limita tamanho
        
        # Monta resposta final
        final_response = f"""
NOTÍCIAS ENCONTRADAS:
{news_result[:500]}...

ANÁLISE DE SENTIMENTO:
{sentiment_result}
"""
        return final_response

# Criar o agente
agent_executor = SimpleNewsAgent(llm, tools)

print("Agente configurado com sucesso!")
print(f"Tools disponíveis: {[tool.name for tool in tools]}")

Agente configurado com sucesso!
Tools disponíveis: ['buscar_noticias', 'analisar_sentimento']


## 4. Implementação

In [110]:
def contar_tokens_aproximado(texto: str) -> int:
    """
    Conta tokens de forma aproximada (1 token ≈ 4 caracteres em português).
    Método alternativo quando transformers não está disponível.
    """
    return len(texto) // 4

def executar_consulta_com_monitoramento(query: str):
    """
    Executa uma consulta no agente e monitora o uso de tokens.
    """
    print(f"\n🔍 Consultando: {query}\n")
    print("="*60)
    
    # Executar a consulta
    resultado = agent_executor.run(query)
    
    # Contar tokens da entrada (usa método aproximado se get_num_tokens falhar)
    try:
        tokens_entrada = llm.get_num_tokens(query)
    except:
        tokens_entrada = contar_tokens_aproximado(query)
    token_monitor.adicionar_operacao("Entrada do usuário", tokens_entrada)
    
    # Contar tokens da saída
    try:
        tokens_saida = llm.get_num_tokens(resultado)
    except:
        tokens_saida = contar_tokens_aproximado(resultado)
    token_monitor.adicionar_operacao("Resposta do agente", tokens_saida)
    
    print("\n" + "="*60)
    print("RESULTADO:")
    print(resultado)
    
    # Mostrar relatório de tokens
    print(token_monitor.obter_relatorio())
    
    return resultado



In [116]:
# Exemplo 1: Buscar notícias sobre tecnologia
query1 = "Jair Bolsonaro"
resultado1 = executar_consulta_com_monitoramento(query1)


🔍 Consultando: Jair Bolsonaro


Buscando notícias...



Analisando sentimento...

RESULTADO:

NOTÍCIAS ENCONTRADAS:
Jair Messias Bolsonaro (Brazilian Portuguese: [ʒaˈiʁ meˈsi.ɐz bowsoˈnaɾu] ⓘ; born 21 March 1955) is a Brazilian politician and former military officer who served as the 38th president of Brazil from 2019 to … 3 days ago · Jair Bolsonaro (born March 21, 1955, Glicério, Brazil) is a Brazilian politician who served as president of Brazil from 2019 to 2023. Former President Jair Bolsonaro is on trial in Brazil for allegedly masterminding an attempted coup after losing the 2022 election to Luiz Inaci...

ANÁLISE DE SENTIMENTO:
NOTÍCIAS: "Jair Bolsonaro em processo por tentativa de golpe após perder eleição" - Various Brazilian Newspapers (e.g. Folha de S. Paulo, O Globo, UOL Notícias)
SENTIMENTO: NEGATIVO
JUSTIFICATIVA: O texto relata que o ex-presidente Jair Bolsonaro está sendo processado por tentativa de golpe após perder a eleição, o que é considerado um ato negativo e contrário à democracia.


RELATÓRIO DE USO DE TOKENS
Entra

## 5. Mais Exemplos de Uso

Você pode testar diferentes consultas abaixo:

In [112]:
# Exemplo 2: Notícias sobre economia
query2 = "Busque notícias sobre a economia brasileira"
resultado2 = executar_consulta_com_monitoramento(query2)


🔍 Consultando: Busque notícias sobre a economia brasileira


Buscando notícias...



Analisando sentimento...

RESULTADO:

NOTÍCIAS ENCONTRADAS:
 Autoridades incautaron 14 toneladas de cocaína en Buenaventura, el mayor decomiso en una década, según destacó el presidente Gustavo Petro. 4 days ago · Las autoridades en medio de un operativo antidrogas, realizaron la incautación de 14 toneladas de cocaína en Buenaventura, ciudad que tiene el principal puerto de Colombia … 4 days ago · Presidente Petro afirmó que esta incautación de cocaína en el puerto del Pacífico colombiano es la más grande que ha realizado la policía de Colombia en la últ...

ANÁLISE DE SENTIMENTO:
NOTICIAS: "14 toneladas de cocaína incautadas em Buenaventura - Presidente Petro anuncia maior decomiso em década" (varios jornais)
SENTIMENTO: POSITIVO
JUSTIFICATIVA: A notícia relata um grande sucesso da luta contra o tráfico de drogas no país, o que é considerado positivo para a sociedade.


RELATÓRIO DE USO DE TOKENS
Entrada do usuário: 7 tokens
Resposta do agente: 254 tokens
Entrada do usuário: 10 token

In [113]:
# Exemplo 3: Notícias sobre esportes
query3 = "Busque notícias recentes sobre a Copa do Mundo"
resultado3 = executar_consulta_com_monitoramento(query3)


🔍 Consultando: Busque notícias recentes sobre a Copa do Mundo


Buscando notícias...



Analisando sentimento...

RESULTADO:

NOTÍCIAS ENCONTRADAS:
 REI Colorado Springs provides outdoor enthusiasts in the Colorado Springs, Colorado, area with top-brand gear and clothing for camping, climbing, cycling, fitness, hiking, skiing, snowboarding … REI Colorado Springs offers top-brand bikes, cycling gear, bike helmets and cycling clothing for road and mountain bike riders. We are also a full-service Colorado Springs bike shop, offering … Sign up for Colorado Springs, CO area outdoor classes & events. Learn camp cooking tips, how to fix your bike,...

ANÁLISE DE SENTIMENTO:
NOTÍCIAS: "REI Colorado Springs oferece equipamentos de alta marca para praticantes de esportes de aventura na região de Colorado Springs, Colorado" - Publicado em diversos jornais locais e de esportes
SENTIMENTO: POSITIVO
JUSTIFICATIVA: A notícia apresenta informações sobre uma loja que oferece uma ampla variedade de equipamentos de alta qualidade para praticantes de esportes de aventura, além de oferecer c

**Desenvolvido como trabalho da disciplina de Inteligência Artificial**

Grupo: Gabriel H, Alan de L, Otavio A, Taiyo T, Matheus Ar.